# NBA 2024-25: Utilizing Roles
## Notebook 02: Data Acquisition, Pt. 2
This notebook collects game-level team statistics for the 2024-25 NBA regular season using the NBA API. It then merges them with game-level player statistics (from Notebook 01), so Usage Rate (USG%) and PRA (Points + Rebounds + Assists) can be calculated at the player-game level. 

In [2]:
# Install NBA API
!pip install nba_api

In [3]:
# Import libraries
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog

In [4]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

___
## Load Player Game Logs

In [6]:
# Load game logs parquets (from Notebook 01)
player_logs = pd.read_parquet(r"C:\Users\dylan\OneDrive\Documents\Portfolio_Projects\NBA_2024_25_utilizing_roles\03_python_outputs\NBA_Player_GameLogs_2024_25.parquet")

___
## Acquire Team Game Logs

In [8]:
# --- Download each team's game logs ---
team_logs = leaguegamelog.LeagueGameLog(
    season="2024-25",
    season_type_all_star="Regular Season"
).get_data_frames()[0]

___
## Clean

In [10]:
# Player logs columns
player_logs.columns

Index(['Player_Name', 'SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE',
       'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'],
      dtype='object')

In [11]:
# Team logs columns
team_logs.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [12]:
# Only keep player columns necessary for USG% and Points-Rebounds-Assists calculations
player_logs_clean = player_logs[[
    "Player_Name", "Player_ID", "Game_ID", "GAME_DATE", "MATCHUP", "WL",
    "MIN", "FGM", "FGA", "FTA", "TOV", "REB", "AST", "PTS"
]].copy()

# Rename Game_ID column so matches the corresponding column in team logs
player_logs_clean = player_logs_clean.rename(columns={
    "Game_ID": "GAME_ID"
})

In [13]:
# Only keep team columns necessary for USG% calculation
team_logs_clean = team_logs[[
    "GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION",
    "MIN", "FGA", "FTA", "TOV"
]].copy()

# Rename columns to emphasize they are team-related
team_logs_clean = team_logs_clean.rename(columns={
    "MIN": "TEAM_MIN",
    "FGA": "TEAM_FGA",
    "FTA": "TEAM_FTA",
    "TOV": "TEAM_TOV"
})

In [14]:
# Extract player's team abbreviation from MATCHUP
player_logs_clean["TEAM_ABBREVIATION"] = player_logs_clean["MATCHUP"].str.split().str[0]

In [15]:
# Confirm abbreviations align between team logs and player logs
set(player_logs_clean["TEAM_ABBREVIATION"]) - set(team_logs_clean["TEAM_ABBREVIATION"])

set()

Empty set ✅

___
## Merge

In [18]:
# Merge player logs with team logs
merged = player_logs_clean.merge(
    team_logs_clean,
    on=["GAME_ID", "TEAM_ABBREVIATION"],
    how="left"
)

In [19]:
# Confirm columns
merged.columns

Index(['Player_Name', 'Player_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FTA', 'TOV', 'REB', 'AST', 'PTS',
       'TEAM_ABBREVIATION', 'TEAM_ID', 'TEAM_MIN', 'TEAM_FGA', 'TEAM_FTA',
       'TEAM_TOV'],
      dtype='object')

___
## Pre-Analysis Calculations

In [21]:
# Calculate USG% for every player-game combination
merged["USG%"] = (
    (merged["FGA"] + 0.44*merged["FTA"] + merged["TOV"]) * (merged["TEAM_MIN"] / 5)
) / (
    merged["MIN"] * (merged["TEAM_FGA"] + 0.44*merged["TEAM_FTA"] + merged["TEAM_TOV"])
)

merged["USG%"] = merged["USG%"] * 100

In [22]:
# Calculate PRA (Points + Rebounds + Assists) for every player-game combination
merged["PRA"] = merged["PTS"] + merged["REB"] + merged["AST"]

In [23]:
# Confirm
merged[["Player_Name", "GAME_DATE", "USG%", "PRA"]].sample(5)

,Player_Name,GAME_DATE,USG%,PRA
11147,James Harden,"Dec 16, 2024",37.521586,48
20476,Pascal Siakam,"Feb 24, 2025",30.514871,30
14531,Kai Jones,"Apr 04, 2025",15.250589,14
13440,Jordan Poole,"Feb 10, 2025",30.738558,26
7476,Duncan Robinson,"Nov 12, 2024",21.081349,23


___
## Save

In [40]:
# Save to parquet
merged.to_parquet("Merged_Player_Team_GameLogs_2024_25.parquet")